### 赤いロボット（捕食者）を追加する

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import mpl_toolkits.mplot3d.axes3d as p3
rc("animation", html="jshtml")

In [28]:
# 反発力：衝突を回避する力
def Fr(X, rr):
    fr = np.array([])
    for i in np.arange(len(X)):
        # i番目のロボット
        fr_i = np.array([])
        for j in np.arange(len(X)):
            if i != j: 
                # Xi-Xj
                molecule = X[i] - X[j] 
                # 反発力が働く場合：反発する
                if np.linalg.norm(molecule) < rr:
                    norm = np.linalg.norm(molecule) ** 2
                    molecule_norm = molecule/norm
                # 反発しない場合：近くにロボットがいない
                else:
                    molecule_norm = np.array([0.0, 0.0, 0.0])
                fr_i = np.append(fr_i, (molecule_norm))
        fr = np.append(fr, fr_i.reshape(len(X)-1, 3).sum(0))
    return fr.reshape(len(X), 3)

# 整列力：同じ方向に移動する力
def Fa(X, V, ra):
    a = V - V[:, None]
    b = (np.linalg.norm(X-X[:, None], axis=2)<ra).astype(np.int)
    return (a * b).sum(0)

# 結合力：群れの中心に集まる力
def Fh(X):
    Xh = X.sum(0) / (len(X)-1)
    return Xh - X

# 抵抗力：移動する際にかかる力
def Ff(V, s=10):
    return -V*(np.linalg.norm(V, axis=1).reshape(len(V), 1) - s)/s

# 回避力：捕食者から襲われた際に逃げる力
def Fp(X, xp, rp):
    fpi = np.array([])
    for i in np.arange(len(X)):
        Xi_Xp = rp - np.linalg.norm(X[i] - xp)
        if Xi_Xp > 0:
            molu_norm_2 = (X[i]-xp) / (np.linalg.norm(X[i]-xp) ** 2)
            # print("回避行動！", molu_norm_2, "Xi", X[i], "xp", xp, Xi_Xp)
        else:
            molu_norm_2 = np.array([0.0, 0.0, 0.0])
            # print("回避しない", "Xi", X[i], "xp", xp, Xi_Xp)
        fpi = np.append(fpi, molu_norm_2)
    return fpi.reshape(len(X), 3)

def F(X, V, xp, Ka=0.1, Kr=1.0, Kf=0.02, Kh=5, Kp=100, a=200, b=0.1, rr=1.0, ra=1.0, rp=2.0):
    F = Kr*Fr(X, rr) + Kh*Fh(X) + Kf*Ff(V) # + Kp*Fp(X, xp, rp)
    return F #a * np.tanh(b * F)

In [20]:
def xp_move(X, xp, Er=3.0):
    # 捕食者がロボットをとらえた場合，一番近いロボットを割り出す．
    l = Er - np.linalg.norm(X - xp, axis=1)[:, None]
    n = 50
    if np.any(l>0):
        # 一番近いロボットに飛びつく
        near_x = np.argmin(np.where(l<0, np.abs(l)+Er, l))
        xp = np.array([
                        np.linspace(xp[0], X[near_x][0]+5, n),
                        np.linspace(xp[1], X[near_x][1]+5, n),
                        np.linspace(xp[2], X[near_x][2]+5, n)
                    ]).reshape(n,3)
    return xp, np.any(l>0)

In [4]:
from scipy.integrate import solve_ivp
def lorenz63(t, arr_xyz, ss, rr, bb):
    # 時刻t-1の値
    x, y, z = arr_xyz
    # Lorenz63モデルのパラメタ
    s, r, b = ss, rr, bb

    # 時刻tの値
    dxdt = -s * (x-y)
    dydt = -x*z + r*x -y
    dzdt = x*y - b*z

    return np.array([dxdt, dydt, dzdt])

# def zscore(x, axis = None):
#     # 平均値    
#     xmean = x.mean(axis=axis, keepdims=True)
#     # xp座標(x,y,z)の標準偏差
#     xstd  = np.std(x, axis=axis, keepdims=True)
#     # 標準化：標準偏差2，平均0
#     zscore = (x-xmean)/xstd
#     return zscore*2

# --- 常微分方程式計算 --- #
# scipy.integrate.solve_ivpでLorenz63モデルを計算
def solver(xp):
    time = 10
    # solver = solve_ivp(fun=lorenz63, t_span=time_range, y0=xp, method=method, t_eval=time_step, dense_output=dense_output, args=lorent_srb)
    solver = solve_ivp(fun=lorenz63, t_span=(0, time), y0=xp, method='RK45', t_eval=np.arange(0, time, 0.02), dense_output=True, args=(10, 26, 8/3))
    # 変数入れ替え
    xyz = solver.y
    # データの形を変える：3,50 → 50,3，転置
    xyz = xyz.T
    # z座標だけ標準化
    # xyz = zscore(xyz[2])

    return xyz

In [12]:
import warnings
warnings.filterwarnings('ignore')

# 3D ローレンツアトラクター

In [ ]:
N = 200
X = np.random.randn(N, 3)*5
V = np.random.randn(N, 3)
dt = 0.02

fig = plt.figure(figsize=(10, 10))
ax = p3.Axes3D(fig)
ims = []

size = 10
ax.set_xlim3d([-size, size])
ax.set_ylim3d([-size, size])
ax.set_zlim3d([-size, size])

# 捕食者の軌跡
xp_tail = np.zeros((10, 3))
r = 5
t = -5
xp_n = np.array([np.cos(np.pi/30*50)*r, np.sin(np.pi/30*50)*r, t])

count = 0
state = 0  # 索敵状態：0, 捕獲状態：1
for t in np.arange(0, dt, dt):
#     if state == 0:
#         xp, check = xp_move(X, xp_n)        
#     if check or state==1: # 捕獲へ推移
#         try:
#             state = 1
#             count += 1
#             xp_n = xp[count]
#         except IndexError:
#             count = 0
#             state = 0        
            
#     else: # 探索状態
    # xp_n = np.array([np.cos(np.pi/30*(t*50))*r, np.sin(np.pi/30*t*50)*r, t])
    # xp_tail = np.append(xp_n, np.delete(xp_tail, np.s_[-3:])).reshape(10, 3)
    
    X = X + V * dt
    V = V + F(X, V, xp_n) * dt
    
    # ims.append([ax.scatter(X[:, 0], X[:, 1], X[:, 2], c="b", s=50),
    #             ax.scatter(xp_n[0], xp_n[1], xp_n[2], c="r", s=50)] + ax.plot(xp_tail[:,0], xp_tail[:,1], xp_tail[:,2], c="slategrey"))
    
    ims.append([ax.scatter(X[:, 0], X[:, 1], X[:, 2], c="b", s=50)])

In [ ]:
animation.ArtistAnimation(fig, ims, interval=50)